# Project Goal:
***The project goal would be an identification of the subway stations that have the most crowded, to help an Ads agency target crowds from busy stations and present its marketing products. For these, I will analyze the visitors of all stations and identify the busiest stations to target. As a result, the crowds will be shown and detected on the live graph based on congestion time.***

## Data Gathering:
**In this section, I will gather the data from [MTA Turnstile dataset](http://web.mta.info/developers/turnstile.html) in the period from August to October of 2021**

In [2]:
# Importing the necessary packages to collect data
import pandas as pd 

#initialise the number of weeks for the dataset
num_weeks = 13

#initialise the date for the first week of the dataset (week ending on start_date)
start_date = filedate = pd.Timestamp('2021-08-07 00:00:00')

#initialise the regex for the MTA turnstile url
filename_regex = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"

filelist = []

for numfiles in range (num_weeks):

    # create the appropriate filename for the week
    filedate_str = str(filedate.year)[2:4] + str(filedate.month).zfill(2) + str(filedate.day).zfill(2)
    filename = filename_regex.format(filedate_str)

    # read the file and append it to the list of files to be concacated
    df = pd.read_csv(filename, parse_dates=['DATE'], keep_date_col=True)
    filelist.append(df)

    # advance to the next week
    filedate += pd.Timedelta(days=7)

mta_test = pd.concat(filelist, axis=0, ignore_index=True)

# Convert Data:
**After gathering the required data, I will save it in CSV format**

In [13]:
mta_test.to_csv('MTA_Dataset.csv', index=False)

# Preparing the data:
**In this section, I'm going to prepare the data I collected previously**

## Field Description:

<br />

| Field Name  |                Description |
| ----------- | ----------- |
| C/A         | Control Area (e.g., A002)              |
| UNIT        | Remote Unit for a station (e.g., R051) |
| SCP      | Subunit Channel Position represents an specific address for a device (e.g., 02-00)|
| STATION     | Represents the station name the device is located at |
| LINENAME    | Represents all train lines that can be boarded at this station |
| DIVISION    | Represents the Line originally the station belonged to BMT, IRT, or IND |
| DATE        | Represents the date (MM-DD-YY) |
| TIME        | Represents the time (hh:mm:ss) for a scheduled audit event |
| DESC        | Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours) |
| ENTRIES     | The comulative entry register value for a device |
| EXITS       | The cumulative exit register value for a device |


In [15]:
# Importing the necessary packages to prepare the data
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime

In [16]:
# Read the data fram
df = pd.read_csv('MTA_Dataset.csv')

In [18]:
# This script will fetch the data and produce an insightful summary
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    return summary

In [19]:
resumetable(df)

Dataset Shape: (2726588, 11)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,C/A,object,0,751,A002,A002,A002
1,UNIT,object,0,469,R051,R051,R051
2,SCP,object,0,224,02-00-00,02-00-00,02-00-00
3,STATION,object,0,379,59 ST,59 ST,59 ST
4,LINENAME,object,0,114,NQR456W,NQR456W,NQR456W
5,DIVISION,object,0,6,BMT,BMT,BMT
6,DATE,object,0,91,2021-07-31,2021-07-31,2021-07-31
7,TIME,object,0,57061,00:00:00,04:00:00,08:00:00
8,DESC,object,0,2,REGULAR,REGULAR,REGULAR
9,ENTRIES,int64,0,1833843,7611181,7611192,7611197


In [ ]:
# station_code = C/A + unit, locating a station
# turnstile = C/A + unit + SCP, locating a turnstile
# datetime = date + time (MM-DD-YY hh:mm:ss)

In [22]:
df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

# Column renaming:
***After I saw the data columns, I notice that the EXITS column of dataset,
<br />
has a whole banch of trailing spaces,
<br />
So to make our data exploration easier I will renamed it :)*** 

In [23]:
df.rename(columns = {'EXITS                                                               ':'EXITS'},
         inplace = True)

In [24]:
df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')